# Annotating Training Data With MTurk

## Pre-requisites
If you haven't already, you'll need to setup MTurk and AWS accounts that are linked together to use MTurk with Python. The MTurk account will be used to post tasks to the MTurk crowd and the AWS accounts will be used to connect to MTurk via API and provide access to any additional AWS resources that are needed to execute your task.

1. If you don't have an AWS account already, visit https://aws.amazon.com and create an account you can use for your project.
2. If you don't have an MTurk Requester account already, visit https://requester.mturk.com and create a new account.

After you've setup your accounts, you will need to link them together. When logged into both the root of your AWS account and your MTurk account, visit https://requester.mturk.com/developer to link them together.

From your AWS console create a new AWS IAM User or select an existing one you plan to use. Add the AmazonMechanicalTurkFullAccess policy to your user. Then select the Security Credentials tab and create a new Access Key, copy the Access Key and Secret Access Key for future use.

If you haven't installed the awscli yet, install it with pip (pip install awscli) and configure a profile using the access key and secret key above (aws configure --profile mturk). 

To post tasks to MTurk for Workers to complete you will first need to add funds to your account that will be used to reward Workers. Visit https://requester.mturk.com/account to get started with as little as $1.00.

We also recommend installing xmltodict as shown below.

In [ ]:
!pip install boto3

In [ ]:
!pip install xmltodict

## Overview
Amazon Mechanical Turk allows you to post tasks for Workers to complete at https://worker.mturk.com. To post a task to
MTurk you create an HTML form that includes the information you want them to provide. In this example we'll be asking Workers to rate the sentiment of Tweets on a scale of 1 (negative) to 10 (positive).

MTurk has a Sandbox environment that can be used for testing. Workers won't work see your tasks in the Sandbox but you can log in to do them yourself to test the task interface at https://workersandbox.mturk.com. It's recommended you test first in the Sandbox to make sure your task returns the data you need before moving to the Production environment. There is no cost to use the Sandbox environment.

In [1]:
import boto3
import xmltodict
import json
import os
from datetime import datetime
import random


In [2]:
create_hits_in_production = False
environments = {
        "production": {
            "endpoint": "https://mturk-requester.us-east-1.amazonaws.com",
            "preview": "https://www.mturk.com/mturk/preview"
        },
        "sandbox": {
            "endpoint": "https://mturk-requester-sandbox.us-east-1.amazonaws.com",
            "preview": "https://workersandbox.mturk.com/mturk/preview"
        },
}
mturk_environment = environments["production"] if create_hits_in_production else environments["sandbox"]

session = boto3.Session(profile_name='mturk')

client = session.client(
    service_name='mturk',
    region_name='us-east-1',
    endpoint_url=mturk_environment['endpoint'],
)

In [3]:
# This will return your current MTurk balance if you are connected to Production.
# If you are connected to the Sandbox it will return $10,000.
print(client.get_account_balance()['AvailableBalance'])

10000.00


## Define your task
For this project we are going to get the sentiment of a set of tweets that we plan to train a model to evaluate. We will create an MTurk Human Intelligence Task (HIT) for each tweet.

In [4]:
imagePath = "https://my-image-repo-520.s3.amazonaws.com/ads"
with open ("ad_filelist.csv","r") as f:
    imageNames = f.readlines()
listOfImages = ["{}/{}".format(imagePath,img.strip().replace(" ","+")) for img in imageNames]


In [5]:
listOfImages

['https://my-image-repo-520.s3.amazonaws.com/ads/Alternet+Page+-+Adidas+Ad.png',
 'https://my-image-repo-520.s3.amazonaws.com/ads/Alternet+Page+-+Impeach+Ad.png',
 'https://my-image-repo-520.s3.amazonaws.com/ads/Alternet+Page+-+Liberal+Quiz+Ad.png',
 'https://my-image-repo-520.s3.amazonaws.com/ads/Alternet+Page+-+NRA+Ad.png',
 'https://my-image-repo-520.s3.amazonaws.com/ads/Alternet+Page+-+Polar+Bear+Ad.png',
 'https://my-image-repo-520.s3.amazonaws.com/ads/Alternet+Page+-+Wallet+Ad.png',
 'https://my-image-repo-520.s3.amazonaws.com/ads/Breitbart+Page+-+Adias+Ad.PNG',
 'https://my-image-repo-520.s3.amazonaws.com/ads/Breitbart+Page+-+NRA+Ad.PNG',
 'https://my-image-repo-520.s3.amazonaws.com/ads/Breitbart+Page+-+Wallet+Ad.PNG',
 'https://my-image-repo-520.s3.amazonaws.com/ads/Breitbart+Page+-+Wildlife+Ad.PNG',
 'https://my-image-repo-520.s3.amazonaws.com/ads/Brietbart+Page+-+Impeach+Ad.PNG',
 'https://my-image-repo-520.s3.amazonaws.com/ads/Brietbart+Page+-+Liberal+Quiz+Ad.PNG',
 'https:/

MTurk accepts an XML document containing the HTML that will be displayed to Workers. Workers will see these HTML for each item tweet that is submitted. To use the HTML for this example task, download it from [here](https://s3.amazonaws.com/mturk/samples/jupyter-examples/SentimentQuestion.html) and store it in the same directory as this notebook. Within the HTML is a variable ${content} that will be replaced with a different tweet when the HIT is created.

Here the HTML is loaded and inserted into the XML Document.

In [6]:
html_layout = open('./survey.html', 'r',encoding="utf-8").read()
QUESTION_XML = """<HTMLQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2011-11-11/HTMLQuestion.xsd">
        <HTMLContent><![CDATA[{}]]></HTMLContent>
        <FrameHeight>650</FrameHeight>
        </HTMLQuestion>"""
question_xml = QUESTION_XML.format(html_layout)

In Mechanical Turk each task is representated by a Human Intelligence Task (HIT) which is an individual item you want annotated by one or more Workers and the interface that should be displayed. The definition below requests that five Workers review each item, that the HIT remain live on the worker.mturk.com website for no more than an hour, and that Workers provide a response for each item in less than ten minutes. Each response has a reward of \$0.05 so the total Worker reward for this task would be \$0.25 plus \$0.05 in MTurk fees. An appropriate title, description, keywords are also provided to let Workers know what is involved in this task.

In [7]:
TaskAttributes = {
    'MaxAssignments': 5,                 
    'LifetimeInSeconds': 60*60,           # How long the task will be available on the MTurk website (1 hour)
    'AssignmentDurationInSeconds': 60*10, # How long Workers have to complete each item (10 minutes)
    'Reward': '0.05',                     # The reward you will offer Workers for each response
    'Title': 'Answer questions about ads',
    'Keywords': 'survey, ad',
    'Description': 'Rate the relevancy of an ad from 1 to 5'
}


In [8]:
random_images = random.choices(listOfImages, k=12)
random_images

['https://my-image-repo-520.s3.amazonaws.com/ads/Alternet+Page+-+NRA+Ad.png',
 'https://my-image-repo-520.s3.amazonaws.com/ads/Alternet+Page+-+NRA+Ad.png',
 'https://my-image-repo-520.s3.amazonaws.com/ads/NYT+Page2+-+NRA+Ad.png',
 'https://my-image-repo-520.s3.amazonaws.com/ads/NYT+Page2+-+Impeach+Ad.png',
 'https://my-image-repo-520.s3.amazonaws.com/ads/Daily+Kos+Page+-+Impeach+Ad.png',
 'https://my-image-repo-520.s3.amazonaws.com/ads/NewsMax+Page+-+Wallet+Ad.png',
 'https://my-image-repo-520.s3.amazonaws.com/ads/Alternet+Page+-+Liberal+Quiz+Ad.png',
 'https://my-image-repo-520.s3.amazonaws.com/ads/Breitbart+Page+-+NRA+Ad.PNG',
 'https://my-image-repo-520.s3.amazonaws.com/ads/NYT+Page+-+Adidas+Ad.png',
 'https://my-image-repo-520.s3.amazonaws.com/ads/NYT+Page2+-+Wallet+Ad.png',
 'https://my-image-repo-520.s3.amazonaws.com/ads/Daily+Kos+Page+-+Adidas+Ad.PNG',
 'https://my-image-repo-520.s3.amazonaws.com/ads/Daily+Kos+Page+-+Wallet+Ad.PNG']

## Create the tasks
Here a HIT is created for each tweet so that it can be completed by Workers. Prior to creating the HIT, the tweet is inserted into the Question XML content. The HIT Id returned for each task is stored in a results array so that we can retrieve the results later.

In [16]:
import random
results = []
hit_type_id = ''
numberOFImage = 12
question = question_xml
for i in range(numberOFImage):
    question = question.replace('${{url_{0}}}'.format(i+1), random_images[i])
    
response = client.create_hit(
    **TaskAttributes,
    Question = question
)
hit_type_id = response['HIT']['HITTypeId']
result = {}
for i in range(numberOFImage):
    result['image{}'.format(i+1)] = random_images[i]
    
    
result['hit_id'] = response['HIT']['HITId']
results.append(result)

print("You can view the HITs here:")
print(mturk_environment['preview'] + "?groupId={}".format(hit_type_id))

if not os.path.exists("result/"):
    os.makedirs("result/")
    
now = datetime.now()

dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
with open('result/result-{}.json'.format(dt_string), 'w') as outfile:
    json.dump(results, outfile)

You can view the HITs here:
https://workersandbox.mturk.com/mturk/preview?groupId=3C7PNR93YGTP8MKRMKX73R97TEGE4I


In [37]:
results

[{'image1': 'https://my-image-repo-520.s3.amazonaws.com/ads/Alternet+Page+-+NRA+Ad.png',
  'image2': 'https://my-image-repo-520.s3.amazonaws.com/ads/Alternet+Page+-+NRA+Ad.png',
  'image3': 'https://my-image-repo-520.s3.amazonaws.com/ads/NYT+Page2+-+NRA+Ad.png',
  'image4': 'https://my-image-repo-520.s3.amazonaws.com/ads/NYT+Page2+-+Impeach+Ad.png',
  'image5': 'https://my-image-repo-520.s3.amazonaws.com/ads/Daily+Kos+Page+-+Impeach+Ad.png',
  'image6': 'https://my-image-repo-520.s3.amazonaws.com/ads/NewsMax+Page+-+Wallet+Ad.png',
  'image7': 'https://my-image-repo-520.s3.amazonaws.com/ads/Alternet+Page+-+Liberal+Quiz+Ad.png',
  'image8': 'https://my-image-repo-520.s3.amazonaws.com/ads/Breitbart+Page+-+NRA+Ad.PNG',
  'image9': 'https://my-image-repo-520.s3.amazonaws.com/ads/NYT+Page+-+Adidas+Ad.png',
  'image10': 'https://my-image-repo-520.s3.amazonaws.com/ads/NYT+Page2+-+Wallet+Ad.png',
  'image11': 'https://my-image-repo-520.s3.amazonaws.com/ads/Daily+Kos+Page+-+Adidas+Ad.PNG',
  'im

## Get Results
Depending on the task, results will be available anywhere from a few minutes to a few hours. Here we retrieve the status of each HIT and the responses that have been provided by Workers.

In [50]:
def getAnsewer(answerDict):
    answer ={}
    for ans in answer_dict['QuestionFormAnswers']['Answer']:
        if ans['QuestionIdentifier'] == "age":
            answer["age"] = ans["FreeText"]
        elif  ans['QuestionIdentifier'] == "gender":
            answer["gender"] = ans["FreeText"]
        elif  ans['QuestionIdentifier'] == "race":
            answer["race"] = ans["FreeText"]
        elif  ans['QuestionIdentifier'] == "zipCode":
            answer["zipCode"] = ans["FreeText"]
        elif  ans['QuestionIdentifier'] == "Hispanic":
            answer["Hispanic"] = ans["FreeText"]
        elif  ans['QuestionIdentifier'] == "education":
            answer["education"] = ans["FreeText"]
        elif  ans['QuestionIdentifier'] == "occupation":
            answer["occupation"] = ans["FreeText"]
                
        elif  ans['QuestionIdentifier'] == "d1.strong_disagree" and ans["FreeText"] == "true":
            answer["feelAboutAd"] = 1
        elif  ans['QuestionIdentifier'] == "d1.disagree" and ans["FreeText"] == "true":
            answer["feelAboutAd"] = 2
        elif  ans['QuestionIdentifier'] == "d1.neutral" and ans["FreeText"] == "true":
            answer["feelAboutAd"] = 3
        elif  ans['QuestionIdentifier'] == "d1.agree" and ans["FreeText"] == "true":
            answer["feelAboutAd"] = 4
        elif  ans['QuestionIdentifier'] == "d1.strong_agree" and ans["FreeText"] == "true":
            answer["feelAboutAd"] = 5
            
        elif  ans['QuestionIdentifier'] == "q1.strong_disagree" and ans["FreeText"] == "true":
            answer["q1"] = 1
        elif  ans['QuestionIdentifier'] == "q1.disagree" and ans["FreeText"] == "true":
            answer["q1"] = 2
        elif  ans['QuestionIdentifier'] == "q1.neutral" and ans["FreeText"] == "true":
            answer["q1"] = 3
        elif  ans['QuestionIdentifier'] == "q1.agree" and ans["FreeText"] == "true":
            answer["q1"] = 4
        elif  ans['QuestionIdentifier'] == "q1.strong_agree" and ans["FreeText"] == "true":
            answer["q1"] = 5
            
        elif  ans['QuestionIdentifier'] == "q2.strong_disagree" and ans["FreeText"] == "true":
            answer["q2"] = 1
        elif  ans['QuestionIdentifier'] == "q2.disagree" and ans["FreeText"] == "true":
            answer["q2"] = 2
        elif  ans['QuestionIdentifier'] == "q2.neutral" and ans["FreeText"] == "true":
            answer["q2"] = 3
        elif  ans['QuestionIdentifier'] == "q2.agree" and ans["FreeText"] == "true":
            answer["q2"] = 4
        elif  ans['QuestionIdentifier'] == "q2.strong_agree" and ans["FreeText"] == "true":
            answer["q2"] = 5
            
        elif  ans['QuestionIdentifier'] == "q3.strong_disagree" and ans["FreeText"] == "true":
            answer["q3"] = 1
        elif  ans['QuestionIdentifier'] == "q3.disagree" and ans["FreeText"] == "true":
            answer["q3"] = 2
        elif  ans['QuestionIdentifier'] == "q3.neutral" and ans["FreeText"] == "true":
            answer["q3"] = 3
        elif  ans['QuestionIdentifier'] == "q3.agree" and ans["FreeText"] == "true":
            answer["q3"] = 4
        elif  ans['QuestionIdentifier'] == "q3.strong_agree" and ans["FreeText"] == "true":
            answer["q3"] = 5
            
        elif  ans['QuestionIdentifier'] == "q4.strong_disagree" and ans["FreeText"] == "true":
            answer["q4"] = 1
        elif  ans['QuestionIdentifier'] == "q4.disagree" and ans["FreeText"] == "true":
            answer["q4"] = 2
        elif  ans['QuestionIdentifier'] == "q4.neutral" and ans["FreeText"] == "true":
            answer["q4"] = 3
        elif  ans['QuestionIdentifier'] == "q4.agree" and ans["FreeText"] == "true":
            answer["q4"] = 4
        elif  ans['QuestionIdentifier'] == "q4.strong_agree" and ans["FreeText"] == "true":
            answer["q4"] = 5
            
        elif  ans['QuestionIdentifier'] == "q5.strong_disagree" and ans["FreeText"] == "true":
            answer["q5"] = 1
        elif  ans['QuestionIdentifier'] == "q5.disagree" and ans["FreeText"] == "true":
            answer["q5"] = 2
        elif  ans['QuestionIdentifier'] == "q5.neutral" and ans["FreeText"] == "true":
            answer["q5"] = 3
        elif  ans['QuestionIdentifier'] == "q5.agree" and ans["FreeText"] == "true":
            answer["q5"] = 4
        elif  ans['QuestionIdentifier'] == "q5.strong_agree" and ans["FreeText"] == "true":
            answer["q5"] = 5
            
        elif  ans['QuestionIdentifier'] == "q6.strong_disagree" and ans["FreeText"] == "true":
            answer["q6"] = 1
        elif  ans['QuestionIdentifier'] == "q6.disagree" and ans["FreeText"] == "true":
            answer["q6"] = 2
        elif  ans['QuestionIdentifier'] == "q6.neutral" and ans["FreeText"] == "true":
            answer["q6"] = 3
        elif  ans['QuestionIdentifier'] == "q6.agree" and ans["FreeText"] == "true":
            answer["q6"] = 4
        elif  ans['QuestionIdentifier'] == "q6.strong_agree" and ans["FreeText"] == "true":
            answer["q6"] = 5
            
        elif  ans['QuestionIdentifier'] == "q7.strong_disagree" and ans["FreeText"] == "true":
            answer["q7"] = 1
        elif  ans['QuestionIdentifier'] == "q7.disagree" and ans["FreeText"] == "true":
            answer["q7"] = 2
        elif  ans['QuestionIdentifier'] == "q7.neutral" and ans["FreeText"] == "true":
            answer["q7"] = 3
        elif  ans['QuestionIdentifier'] == "q7.agree" and ans["FreeText"] == "true":
            answer["q7"] = 4
        elif  ans['QuestionIdentifier'] == "q7.strong_agree" and ans["FreeText"] == "true":
            answer["q7"] = 5
            
        elif  ans['QuestionIdentifier'] == "q8.strong_disagree" and ans["FreeText"] == "true":
            answer["q8"] = 1
        elif  ans['QuestionIdentifier'] == "q8.disagree" and ans["FreeText"] == "true":
            answer["q8"] = 2
        elif  ans['QuestionIdentifier'] == "q8.neutral" and ans["FreeText"] == "true":
            answer["q8"] = 3
        elif  ans['QuestionIdentifier'] == "q8.agree" and ans["FreeText"] == "true":
            answer["q8"] = 4
        elif  ans['QuestionIdentifier'] == "q8.strong_agree" and ans["FreeText"] == "true":
            answer["q8"] = 5
            
        elif  ans['QuestionIdentifier'] == "q9.strong_disagree" and ans["FreeText"] == "true":
            answer["q9"] = 1
        elif  ans['QuestionIdentifier'] == "q9.disagree" and ans["FreeText"] == "true":
            answer["q9"] = 2
        elif  ans['QuestionIdentifier'] == "q9.neutral" and ans["FreeText"] == "true":
            answer["q9"] = 3
        elif  ans['QuestionIdentifier'] == "q9.agree" and ans["FreeText"] == "true":
            answer["q9"] = 4
        elif  ans['QuestionIdentifier'] == "q9.strong_agree" and ans["FreeText"] == "true":
            answer["q9"] = 5
            
        elif  ans['QuestionIdentifier'] == "q10.strong_disagree" and ans["FreeText"] == "true":
            answer["q10"] = 1
        elif  ans['QuestionIdentifier'] == "q10.disagree" and ans["FreeText"] == "true":
            answer["q10"] = 2
        elif  ans['QuestionIdentifier'] == "q10.neutral" and ans["FreeText"] == "true":
            answer["q10"] = 3
        elif  ans['QuestionIdentifier'] == "q10.agree" and ans["FreeText"] == "true":
            answer["q10"] = 4
        elif  ans['QuestionIdentifier'] == "q10.strong_agree" and ans["FreeText"] == "true":
            answer["q10"] = 5
            
        elif  ans['QuestionIdentifier'] == "q11.strong_disagree" and ans["FreeText"] == "true":
            answer["q11"] = 1
        elif  ans['QuestionIdentifier'] == "q11.disagree" and ans["FreeText"] == "true":
            answer["q11"] = 2
        elif  ans['QuestionIdentifier'] == "q11.neutral" and ans["FreeText"] == "true":
            answer["q11"] = 3
        elif  ans['QuestionIdentifier'] == "q11.agree" and ans["FreeText"] == "true":
            answer["q11"] = 4
        elif  ans['QuestionIdentifier'] == "q11.strong_agree" and ans["FreeText"] == "true":
            answer["q11"] = 5
            
        elif  ans['QuestionIdentifier'] == "q12.strong_disagree" and ans["FreeText"] == "true":
            answer["q12"] = 1
        elif  ans['QuestionIdentifier'] == "q12.disagree" and ans["FreeText"] == "true":
            answer["q12"] = 2
        elif  ans['QuestionIdentifier'] == "q12.neutral" and ans["FreeText"] == "true":
            answer["q12"] = 3
        elif  ans['QuestionIdentifier'] == "q12.agree" and ans["FreeText"] == "true":
            answer["q12"] = 4
        elif  ans['QuestionIdentifier'] == "q12.strong_agree" and ans["FreeText"] == "true":
            answer["q12"] = 5
    return answer

In [53]:
# with open('result/result-10-11-2019 15:51:20.json', 'r') as f:
#     results = json.load(f.read())
    
for item in results:
    
    # Get the status of the HIT
    hit = client.get_hit(HITId=item['hit_id'])
    item['status'] = hit['HIT']['HITStatus']

    # Get a list of the Assignments that have been submitted by Workers
    assignmentsList = client.list_assignments_for_hit(
        HITId=item['hit_id'],
        AssignmentStatuses=['Submitted', 'Approved'],
        MaxResults=100
    )

    assignments = assignmentsList['Assignments']
    item['assignments_submitted_count'] = len(assignments)

    answers = []
    for assignment in assignments:
    
        # Retreive the attributes for each Assignment
        worker_id = assignment['WorkerId']
        assignment_id = assignment['AssignmentId']
        
        # Retrieve the value submitted by the Worker from the XML
        answer_dict = xmltodict.parse(assignment['Answer'])
#         print(answer_dict)
        answer = getAnsewer(answer_dict)
#         print (answer)
        answers.append(answer)
        
        # Approve the Assignment (if it hasn't already been approved)
        if assignment['AssignmentStatus'] == 'Submitted':
            client.approve_assignment(
                AssignmentId=assignment_id,
                OverrideRejection=False
            )
    
    # Add the answers that have been retrieved for this item
    item['answers'] = answers
#     if len(answers) > 0:
#         item['avg_answer'] = sum(answers)/len(answers)

print(json.dumps(results,indent=2))

[
  {
    "image1": "https://my-image-repo-520.s3.amazonaws.com/ads/Alternet+Page+-+NRA+Ad.png",
    "image2": "https://my-image-repo-520.s3.amazonaws.com/ads/Alternet+Page+-+NRA+Ad.png",
    "image3": "https://my-image-repo-520.s3.amazonaws.com/ads/NYT+Page2+-+NRA+Ad.png",
    "image4": "https://my-image-repo-520.s3.amazonaws.com/ads/NYT+Page2+-+Impeach+Ad.png",
    "image5": "https://my-image-repo-520.s3.amazonaws.com/ads/Daily+Kos+Page+-+Impeach+Ad.png",
    "image6": "https://my-image-repo-520.s3.amazonaws.com/ads/NewsMax+Page+-+Wallet+Ad.png",
    "image7": "https://my-image-repo-520.s3.amazonaws.com/ads/Alternet+Page+-+Liberal+Quiz+Ad.png",
    "image8": "https://my-image-repo-520.s3.amazonaws.com/ads/Breitbart+Page+-+NRA+Ad.PNG",
    "image9": "https://my-image-repo-520.s3.amazonaws.com/ads/NYT+Page+-+Adidas+Ad.png",
    "image10": "https://my-image-repo-520.s3.amazonaws.com/ads/NYT+Page2+-+Wallet+Ad.png",
    "image11": "https://my-image-repo-520.s3.amazonaws.com/ads/Daily+Kos+